# kansverdelingen berekenen

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
df = pd.read_csv('data/adm_clean.csv, sep = ',')
# maak de datum naar een datum type
df['datum_opname'] = pd.to_datetime(df['datum_opname'], errors='coerce')
df['datum_ontslaan'] = pd.to_datetime(df['datum_ontslaan'], errors='coerce')

df

,datum_opname,datum_ontslaan,gepland,ziekte,categorie,verblijfstijd
0,2015-01-01 07:36:00,2015-01-02 16:43:00,unplanned,NEC,4,1 days 09:07:00.000000000
1,2015-01-01 14:23:59,2015-01-01 19:47:00,unplanned,INT,2,0 days 05:23:01.000000000
2,2015-01-01 20:30:59,2015-01-02 19:09:59,unplanned,INT,2,0 days 22:39:00.000000000
3,2015-01-02 12:56:00,2015-01-03 11:32:00,planned,CAPU,1,0 days 22:36:00.000000000
4,2015-01-02 13:12:00,2015-01-03 10:02:59,unplanned,NEC,4,0 days 20:50:59.000000000
...,...,...,...,...,...,...
4478,2016-12-30 18:24:00,2016-12-30 00:00:00,unplanned,INT,2,0 days 12:00:00.000000000
4479,2016-12-30 18:42:59,2016-12-31 00:00:00,planned,CAPU,1,0 days 05:17:01.000000000
4480,2016-12-30 23:19:00,2017-01-01 00:00:00,unplanned,NEC,4,1 days 00:41:00.000000000
4481,2016-12-31 15:29:59,2017-01-01 00:00:00,unplanned,INT,2,0 days 08:30:01.000000000


# instroom

## instroom geplande patienten

In [12]:
df_planned = df.where(df['gepland'] == 'planned').dropna()

df_planned['weekdag'] = df_planned['datum_opname'].dt.day_name()

In [13]:
# berekend de deltas opname tijd tussen de patienten.
def calc_delta_time_vars(data):
    data.sort_values('datum_opname')
    data['opnametijd_delta_now'] = (data['datum_opname'] - pd.datetime.now())
    data['opnametijd_delta_now_hr'] = data['opnametijd_delta_now'].dt.total_seconds() / 3600
    data['opname_delta_hr'] = (data['opnametijd_delta_now_hr']-data['opnametijd_delta_now_hr'].shift()).fillna(0)
    data = data.drop(['opnametijd_delta_now', 'opnametijd_delta_now_hr'], axis=1)
    return data

df_planned = calc_delta_time_vars(df_planned)

df_planned

D:\program files\anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


,datum_opname,datum_ontslaan,gepland,ziekte,categorie,verblijfstijd,weekdag,opname_delta_hr
3,2015-01-02 12:56:00,2015-01-03 11:32:00,planned,CAPU,1.0,0 days 22:36:00.000000000,Friday,0.000000
5,2015-01-02 14:06:00,2015-01-03 10:21:00,planned,CAPU,1.0,0 days 20:15:00.000000000,Friday,1.166667
6,2015-01-02 17:26:00,2015-01-04 13:50:59,planned,CHIR,3.0,1 days 20:24:59.000000000,Friday,3.333333
7,2015-01-02 20:10:00,2015-01-05 10:09:59,planned,CAPU,1.0,2 days 13:59:59.000000000,Friday,2.733333
12,2015-01-03 22:59:00,2015-01-27 15:09:59,planned,INT,2.0,23 days 16:10:59.000000000,Saturday,26.816667
...,...,...,...,...,...,...,...,...
4468,2016-12-28 16:25:00,2017-01-01 00:00:00,planned,CAPU,1.0,3 days 07:35:00.000000000,Wednesday,27.883611
4469,2016-12-29 12:40:00,2016-12-30 00:00:00,planned,CAPU,1.0,0 days 11:20:00.000000000,Thursday,20.250000
4472,2016-12-30 00:22:00,2017-01-03 00:00:00,planned,CAPU,1.0,3 days 23:38:00.000000000,Friday,11.700000
4476,2016-12-30 12:56:00,2016-12-31 00:00:00,planned,CAPU,1.0,0 days 11:04:00.000000000,Friday,12.566667


In [30]:
print('gemiddelde opname deltas per weekdag\n')
for day in df_planned['weekdag'].unique():
    weekday_mean = df_planned['opname_delta_hr'].where(df_planned['weekdag'] == day).mean()
    print(f' {day}: {weekday_mean}')
    


gemiddelde opname deltas per weekdag

 Friday: 8.705045970266005
 Saturday: 19.65161290322571
 Monday: 17.94640769230795
 Tuesday: 10.122215470678965
 Thursday: 7.472981164925745
 Wednesday: 10.820380658435843
 Sunday: 31.28867424242344


In [41]:
kansverdelingen = {}
days = ['maandag', 'dinsdag', 'woensdag', 'donderdag', 'vrijdag', 'zaterdag', 'zondag'] 


for day in df_planned['weekdag'].unique():
    patient_amount = {'CAPU': 0, 'CHIR': 0, 'CARD': 0, 'INT': 0, 'NEC': 0, 'NEU': 0, 'OTHER': 0}
    
    for illness in patient_amount.keys():
        patient_amount[illness] = df_planned['afdeling'].where((df_planned['afdeling'] == illness) & (df_planned['weekdag'] == day)).count()
    print(f'{day}: {patient_amount}')

Friday: {'CAPU': 252, 'CHIR': 14, 'CARD': 2, 'INT': 3, 'NEC': 12, 'NEU': 1, 'OTHER': 0}
Saturday: {'CAPU': 18, 'CHIR': 4, 'CARD': 1, 'INT': 4, 'NEC': 3, 'NEU': 1, 'OTHER': 0}
Monday: {'CAPU': 281, 'CHIR': 28, 'CARD': 0, 'INT': 2, 'NEC': 11, 'NEU': 3, 'OTHER': 0}
Tuesday: {'CAPU': 240, 'CHIR': 41, 'CARD': 0, 'INT': 4, 'NEC': 2, 'NEU': 1, 'OTHER': 0}
Thursday: {'CAPU': 327, 'CHIR': 19, 'CARD': 0, 'INT': 0, 'NEC': 4, 'NEU': 1, 'OTHER': 0}
Wednesday: {'CAPU': 186, 'CHIR': 11, 'CARD': 1, 'INT': 4, 'NEC': 12, 'NEU': 1, 'OTHER': 1}
Sunday: {'CAPU': 11, 'CHIR': 3, 'CARD': 2, 'INT': 2, 'NEC': 3, 'NEU': 1, 'OTHER': 0}
(11, 3, 2, 2, 3, 1, 0)
